In [16]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

output_directory = '/home/marco/DataspellProjects/thesis/output/knn/ST001937/'

palette = sns.color_palette(["#2176AB", "#F97662", "#FFBF00", "#50C878", "#B284BE"])
#create a gradient color palette 
gradient_palette = sns.blend_palette(['#ECF7FE', palette[0]], as_cmap=True)
sns.set(style="whitegrid", font_scale=1.5)
sns.set_palette(palette)

df = pd.read_csv('../../../data/ST001937_bm.csv')
df.describe()

1,3,5(10)-estratrien-3,6- beta-17-beta-triol  1,5-anhydroglucitol  \
count                                    600.000000           600.000000   
mean                                       0.060028             0.081831   
std                                        1.149557             1.157614   
min                                      -10.056505            -7.753742   
25%                                       -0.231721            -0.132294   
50%                                       -0.191786             0.010058   
75%                                       -0.079706             0.151365   
max                                       10.238873            21.620913   

       17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-1  \
count                                     600.000000   
mean                                        0.121458   
std                                         1.158775   
min                                        -1.103028   
25%                                        -0.417097   
50%                                        -0.160029   
75%                                         0.273505   
max                                        12.565668   

       17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-2  \
count                                     600.000000   
mean                                        0.069002   
std                                         1.177830   
min                                        -2.246114   
25%                                        -0.395316   
50%                                        -0.288463   
75%                                         0.039505   
max                                        12.303188   

       17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-3  \
count                                     600.000000   
mean                                        0.153339   
std                                         1.145922   
min                                        -0.763127   
25%                                        -0.624346   
50%                                        -0.094491   
75%                                         0.463881   
max                                        11.517330   

       17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-4  \
count                                     600.000000   
mean                                       -0.028114   
std                                         1.348227   
min                                       -16.350768   
25%                                        -0.018728   
50%                                         0.009537   
75%                                         0.044281   
max                                        18.100453   

       17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-5  \
count                                     600.000000   
mean                                        0.146952   
std                                         1.291587   
min                                        -0.914859   
25%                                        -0.455456   
50%                                        -0.084182   
75%                                         0.381119   
max                                        19.023871   

       17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-6  1-hexadecanol  \
count                                     600.000000     600.000000   
mean                                        0.141551       0.185913   
std                                         1.301322       1.262072   
min                                        -1.005143      -1.729810   
25%                                        -0.427082      -0.343499   
50%                                        -0.116827      -0.006010   
75%                                         0.308743       0.305460   
max                                        14.488387      13.043171   

       1-hydroxyanthraquinone prod  ...  tyrosine-2      urea-1      urea-2  \
count                   600.000000  ...  600.000000  600.000000  600.000000   

Performing PLSDA on the dataset using the roc_auc scoring metric and 10-fold cross validation.

In [17]:
from sklearn.metrics import roc_auc_score

X = df.drop(columns=['Phenotypes'])
y = df['Phenotypes']


In [18]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV


# Definisci il modello
model = KNeighborsClassifier()

# Definisci la griglia dei parametri da esplorare
param_grid = {
    'n_neighbors': range(1, 15, 1),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
}

# Esegui la ricerca esaustiva dei parametri
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=10, n_jobs=-1, verbose=2, scoring='accuracy')
grid_search.fit(X, y)

# Stampa i migliori parametri trovati
print("Migliori parametri:")
print(grid_search.best_params_)
print(grid_search.best_score_)

Fitting 10 folds for each of 84 candidates, totalling 840 fits
Migliori parametri:
{'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'uniform'}
0.835


In [19]:
from sklearn.metrics import accuracy_score, roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold


cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
best_params = grid_search.best_params_
score = {'accuracy': [], 'roc_auc': [], 'fpr': [], 'tpr': []}


for train_index, test_index in cv.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = KNeighborsClassifier(**best_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_prob, drop_intermediate=False)

    score['accuracy'].append(accuracy_score(y_test, y_pred))
    score['roc_auc'].append(roc_auc_score(y_test, y_prob))
    score['fpr'].append(fpr.tolist())
    score['tpr'].append(tpr.tolist())



In [20]:
scores_df = pd.read_csv('../../../output/scores.csv')

scores_df = scores_df.append({
    'Model': 'KNN',
    'Dataset': 'ST001937',
    'auc': score['roc_auc'],
    'accuracy': score['accuracy'],
    'fpr': score['fpr'],
    'tpr': score['tpr']
}, ignore_index=True)
scores_df.to_csv('../../../output/scores.csv', index=False)

/tmp/ipykernel_1254015/3325872913.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scores_df = scores_df.append({


In [21]:
#Confusion Matrix

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = KNeighborsClassifier(**best_params)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

conf_matrix = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)

print(f'Accuracy: {accuracy:.2f}')
print(f'AUC: {auc:.2f}')


plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap=gradient_palette, xticklabels=['Benign', 'Malignant'], yticklabels=['Benign', 'Malignant'])
plt.xlabel('Predicted', labelpad=20)
plt.ylabel('Actual', labelpad=20)
plt.title('Confusion Matrix', pad=20, fontsize=20)
plt.savefig(output_directory + 'knn_confusion_matrix.png')

Accuracy: 0.83
AUC: 0.83
